In [1]:
%pip install gurobipy

import gurobipy as gp
from gurobipy import GRB

You should consider upgrading via the '/Users/elifnurdogruoz/.pyenv/versions/3.9.7/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Resource and job sets
R = ['R1', 'R2', 'R3']
J = ['J1', 'J2', 'J3']

In [3]:
# Matching score data
combinations, scores = gp.multidict({
    ('R1', 'J1'): 53,
    ('R2', 'J2'): 27,
    ('R1', 'J3'): 13,
    ('R2', 'J1'): 80,
    ('R2', 'J2'): 47,
    ('R2', 'J3'): 67,
    ('R3', 'J1'): 53,
    ('R3', 'J2'): 73,
    ('R3', 'J3'): 47
})

In [4]:
# Declare and initialize model
m = gp.Model('RAP')

Set parameter Username
Set parameter LicenseID to value 2604515


In [5]:
# Create decision variables for the RAP model
x = m.addVars(combinations, name="assign")

In [6]:
# Create job constraints
jobs = m.addConstrs((x.sum('*',j) == 1 for j in J), name='job')

In [7]:
# Create resource constraints
resources = m.addConstrs((x.sum(r,'*') <= 1 for r in R), name='resource')

In [8]:
# Objective: maximize total matching score of all assignments
m.setObjective(x.prod(scores), GRB.MAXIMIZE)

In [9]:
# Save model for inspection
#m.write('RAP.lp')

In [10]:
# Run optimization engine
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 24.3.0 24D81)

CPU model: Intel(R) Core(TM) i7-8850H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 6 rows, 8 columns and 16 nonzeros
Model fingerprint: 0x1802ac9b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s
Presolved: 5 rows, 7 columns, 14 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2000000e+02   1.000000e+00   0.000000e+00      0s
       2    1.9300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.930000000e+02


In [11]:
# Display optimal values of decision variables
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal total matching score
print('Total matching score: ', m.objVal)

assign[R1,J1] 1.0
assign[R2,J3] 1.0
assign[R3,J2] 1.0
Total matching score:  193.0
